In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import os, sys

#from arrow import get
sys.path.append('/home/hrai/codes/hpe_library')
from lib_import import *
from my_utils import BatchCamera, Camera
from my_utils import arg_as_list, get_configs
from my_utils import generate_random_segment, generate_random_trajectory, get_aligned_init_torso, get_ap_pose_2d, get_backbone_line_from_torso, get_bounded_segments_idx, get_cam_param, get_input, get_label, get_model_input, get_output, get_pairs, get_part_traj, get_pose_seq_and_cam_param, get_two_point_parts, load_h36m, load_segment_file_from_parameters, make_input, make_one_dimension_list, make_output, split_continuous_indices, MyCustomDataset, parse_args_by_model_name, get_input_gt_for_onevec, get_limb_angle, get_h36m_camera_info, h36m_data_processing
from my_utils import DH_matrix, azim_elev_to_vec, batch_azim_elev_to_vec, batch_inverse_tf, batch_projection, batch_rot_x_matrix, batch_rot_y_matrix, batch_rot_z_matrix, calculate_azimuth_elevation, calculate_rotation_quaternion, draw_arm, draw_subline, draw_torso_direction, frame_vec_to_matrix, generate_batch_tf_from_batch_origin_R, generate_camera_frame, generate_dh_frame, generate_tf_from_origin_R, generate_two_link, generate_world_frame, get_batch_frame_vec_from_keypoints, get_batch_lower_torso_frame_from_keypoints, get_batch_lower_torso_frame_from_pose, get_batch_reference_frame, get_batch_upper_torso_frame_from_keypoints, get_batch_upper_torso_frame_from_pose, get_frame_from_keypoints, get_lower_torso_frame_from_pose, get_reference_frame, get_torso_direction, get_torso_rotation_matrix, get_torso_shape, get_upper_torso_frame_from_pose, inverse_tf, normalize_vector, project_batch_tensor, projection, rotate_torso_by_R, rotate_torso_by_R_for_batch_tensor, rotation_matrix_to_vector_align, rotation_matrix_torso2torso, Appendage, BatchAppendage, BatchDHModel, DHModel, calculate_batch_azimuth_elevation
from my_utils import args_dict_to_namespace, construct_torso_from_output, denormalize_motionbert_result, get_dataset_info_from_segment_folder, get_inference_from_dhdst, get_inference_from_dhdst_torso, get_inference_from_motionbert, get_output_type, get_result, infer_one_segment, load_best_model_for_inference, normalize_input, test_model_by_segment_file
from my_utils import get_logger, log_configs
from my_utils import init_weights, split_array_by_idxs, BaselineModel, Linear, TorsoModel
from my_utils import Camera2ImageCoordinate, MPJPE, MPJPE_for_multiple_pose, MPJPE_for_single_pose, World2CameraCoordinate, World2ImageCoordinate, _sqrt_positive_part, _weak_project, aihub2h36m, array2dict, avgErrorForOneAction, avgErrorForOneActor, avgErrorForOneCamera, camera_to_image_frame, change_bbox_convention, check_max_min, coco2h36m, convert3DResult, draw_skeleton, draw_skeleton_2d, draw_skeleton_both, euclidean_distance, fit3d2h36m, fit3d_load_gt_and_param, get3DResult, getAIHubCameraParameter, getGT, getNumFromImgName, get_batch_h36m_keypoints, get_bbox_area, get_bbox_area_from_pose2d, get_bbox_from_pose2d, get_h36m_keypoint_index, get_h36m_keypoints, get_rootrel_pose, get_video_frame, get_video_info, get_xy_centered_pose, halpe2h36m, infer_box, kookmin2h36m, kookmin2h36m_with_nose, loadAIHubCameraParameter, matrix_to_quaternion, normalize, normalize_array, optimize_scaling_factor, plot_cv2_image, readJSON, readpkl, savepkl, skew_symmetric_matrix, skew_symmetric_matrix_tensor, standardize_quaternion, mpi_inf_3dhp2h36m, get_pose_height, get_batch_bbox_from_pose2d, len_ids, part_ids, h36m_part_keypoints, get_h36m_limb_lens, procrustes_align
from my_utils import check_duplicate_training, get_input_output_candidate, get_num_trial, load_args, load_best_model, load_dataset, load_model, prepare_training, run, run_epoch, save_args, split_array_by_idxs
from my_utils import axes_2d, axes_3d, axes_to_compare_pred_gt, draw_3d_pose, draw_bbox, draw_multiple_2d_pose, draw_multiple_3d_pose, draw_one_segment, draw_rotation_matrix, draw_segment, draw_segments, draw_trajectory, get_2d_pose_image, legend_without_duplicate_labels, plot_to_compare_pred_gt, save_h36m_pose_video, show2Dtrajectory, show3Dtrajectory, show_2d_3d, show_whole_segment_trajectories, draw_2d_pose, clear_axes
from my_utils import check_available_frame, check_continuity, draw_base_marker_3d, generate_kookmin_pkl_for_each_video, get_cam_param_kookmin, get_video_frame_kookmin, get_video_num_frame_kookmin, load_pose3d_kookmin, load_csv_kookmin, get_lbot
os.chdir('/home/hrai/codes/MotionBERT')

from lib.utils.tools import * # get_config
from lib.utils.learning import * # load_backbone
from lib.utils.args import parse_args, check_args
from lib.utils.utils_data import flip_data
from lib.model.loss import *
from lib.model.evaluation import *
from lib.data.datareader_kookmin import DataReaderKOOKMIN
from lib.data.datareader_h36m import DataReaderH36M
from lib.data.datareader_random_limb import DataReaderRandomLimb
from lib.data.dataset_motion_3d import MotionDataset3D
from lib.model.DHDSTformer import DHDSTformer_total, DHDSTformer_total2, DHDSTformer_total3, DHDSTformer_limb, DHDSTformer_torso, DHDSTformer_torso2, DHDSTformer_total4

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output
import threading
from ipywidgets import GridspecLayout
from ipywidgets import TwoByTwoLayout

from utils_result_analysis.button import get_inference_button, get_visualize_button, get_root_rel_button, get_reset_button, get_go_to_max_frame_button, get_procrustes_button, get_analysis_error_button, get_analysis_dh_button
from utils_result_analysis.progress import get_inference_progress
from utils_result_analysis.text import get_inputs_all_shape_text, get_gts_all_shape_text, get_results_all_shape_text, get_error_max_frame_text, get_batch_num_text, get_frame_num_text
from utils_result_analysis.slider import get_azim_slider, get_elev_slider, get_zoom_slider, get_delay_slider, get_frame_slider
from utils_result_analysis.dropdown import get_model_list_dropdown, get_dataset_list_dropdown
from utils_result_analysis.play import get_play_vis_button
from utils_result_analysis.select import get_subject_select, get_action_select, get_cam_select, get_batch_select, get_part_select, get_part_select2

box_layout = widgets.Layout(
        border='solid 1px red',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px')
plt.clf()
plt.cla()

class analysis_dataset:
    def __init__(self):
        self.right_arm_id = 0
        self.left_arm_id = 1
        self.right_leg_id = 2
        self.left_leg_id = 3
        self.test_out = widgets.Output()
        try:
            # model, data preparation -----------------------------------------------------
            self.load_panel()
            # select ---------------------------------------------------------------------
            self.select_panel()
            # figure ---------------------------------------------------------------------
            self.figure_panel()
            # interactive -----------------------------------------------------------------
            self.interactive()
            # layout ----------------------------------------------------------------------
            self.layout()
            # main
            self.main()
        except Exception as e:
            #with self.test_out:
            print(e)

    # panel functions -------------------------------------------------------------------  
    def load_panel(self):
        self.dataset_list = get_dataset_list_dropdown(['H36M'])

    def select_panel(self):
        self.select_subject = get_subject_select([''])
        self.select_action = get_action_select([''])
        #self.select_cam = get_cam_select([''])
        #self.select_batch = get_batch_select([''])
        self.select_subject.layout.width = 'max-content'
        self.select_subject.rows = 7
        self.select_action.layout.width = 'max-content'
        self.select_action.rows = 7
        #self.select_cam.layout.width = 'max-content'
        #self.select_batch.layout.width = 'max-content'
        self.select_part = get_part_select2(['right_arm', 'left_arm', 'right_leg', 'left_leg'])
        self.select_part.layout.width = 'max-content'

    def figure_panel(self):
        self.plot_3d = widgets.Output(layout=box_layout)
        with self.plot_3d:
            self.fig_3d = plt.figure(1, figsize=(15, 15), layout='constrained')
            self.fig_3d.clear()
            self.fig_3d.tight_layout()
            self.ax_3d1 = axes_3d(self.fig_3d, [0, 0, 1, 1], xlim=(-2,2), ylim=(-2,2), zlim=(0,2), view=(10,-90), show_axis=False)      
            self.ax_3d1.set_aspect('equal')    
            f = plt.show()
        self.frame_slider = get_frame_slider(1)
        self.frame_slider.layout.width = 'auto'
        self.frame_button = get_frame_num_text()
        self.frame_button.layout.width = 'auto'
        widgets.jslink((self.frame_slider, 'value'), (self.frame_button, 'value'))

        self.plot_angle = widgets.Output(layout=box_layout)
        with self.plot_angle:
            self.fig_angle = plt.figure(2, figsize=(15,5))
            self.fig_angle.clear()
            #self.fig_angle.tight_layout()
            self.ax_angle = self.fig_angle.add_subplot(111)  
            #self.ax_angle.set_aspect('equal')    
            f = plt.show()

    def load_data(self, target_dataset):
        self.target_dataset = target_dataset
        if target_dataset == 'H36M':
            self.world_3d, self.h36m_cam_param = load_h36m()
            self.subject_list = list(self.world_3d._data.keys())
            self.select_subject.options = self.subject_list
            if self.select_subject.value == '':
                self.select_subject.value = self.select_subject.options[0]
            self.select_action.options = list(self.world_3d._data[self.select_subject.value].keys())
            if self.select_action.value == '':
                self.select_action.value = self.select_action.options[0]
            self.frame_slider.max = self.world_3d._data[self.select_subject.value][self.select_action.value]['positions'].shape[0]
            self.frame_button.max = self.frame_slider.max
            print(f'{target_dataset} Loaded!')
        else:
            print(f'{target_dataset} is not implemented yet')

    def update_select(self, subject):
        try:
            self.select_action.options = list(self.world_3d._data[subject].keys())
            self.select_action.value = self.select_action.options[0]
        except Exception as e:
            with self.test_out:
                print('select error', e)

    def update_pose_3d(self, subject, action):
        with self.test_out:
            print(f'update_pose_3d {subject} {action}')
        try:
            # update pose 3d list
            self.pose_3d_list = self.world_3d._data[self.select_subject.value][action]['positions'].copy()
            # generate dh model
            self.dh_model = DHModel(self.pose_3d_list[0].copy(), forward_dir='x', use_global_frame=False)
            # update frame slider
            self.frame_slider.max = self.pose_3d_list.shape[0]
            self.frame_button.max = self.frame_slider.max
            # get dh angles of all frame
            self.dh_angles = []
            # self.right_arm_link1_azims = []
            # self.right_arm_link2_azims = []
            for i in range(0, len(self.pose_3d_list)):
                pose_3d = self.pose_3d_list[i]
                self.dh_model.update_dh_model_from_pose(pose_3d)
                self.dh_angles.append(self.dh_model.get_dh_angles(by_dict=False, degrees=False)) # (16,)
                # self.right_arm_link1_azims.append(self.dh_model.right_arm.link1_azim)
                # self.right_arm_link2_azims.append(self.dh_model.right_arm.link2_azim)

                assert self.dh_angles[-1][0] == self.dh_model.right_arm.link1_azim, f"right_arm link1 azim mismatch {self.dh_angles[-1][0]:.2f} {self.dh_model.right_arm.link1_azim:.2f}"
                assert self.dh_angles[-1][1] == self.dh_model.right_arm.link1_elev, f"right_arm link1 elev mismatch {self.dh_angles[-1][1]:.2f} {self.dh_model.right_arm.link1_elev:.2f}"
                assert self.dh_angles[-1][2] == self.dh_model.right_arm.link2_azim, f"right_arm link2 azim mismatch {self.dh_angles[-1][2]:.2f} {self.dh_model.right_arm.link2_azim:.2f}"
                assert self.dh_angles[-1][3] == self.dh_model.right_arm.link2_elev, f"right_arm link2 elev mismatch {self.dh_angles[-1][3]:.2f} {self.dh_model.right_arm.link2_elev:.2f}"
            self.dh_angles = np.array(self.dh_angles) # (frame, 16)
            self.dh_angles_diff = self.dh_angles.copy()[:-1] - self.dh_angles.copy()[1:] # (frame-1, 16)
            # self.right_arm_link1_azims = np.array(self.right_arm_link1_azims)
            # self.right_arm_link2_azims = np.array(self.right_arm_link2_azims)
            # self.right_arm_link1_azims_diff = self.right_arm_link1_azims.copy()[:-1] - self.right_arm_link1_azims.copy()[1:]
            # self.right_arm_link2_azims_diff = self.right_arm_link2_azims.copy()[:-1] - self.right_arm_link2_azims.copy()[1:]
            #self.noise_frames = np.where(self.dh_angles_diff > 1)

            self.dh_model = DHModel(self.pose_3d_list[0].copy(), forward_dir='x', use_global_frame=False)
            #self.dh_model.update_dh_model_from_pose(self.pose_3d_list[0])
            self.visualize_angle(self.select_part.value)
            self.update_frame_data(self.frame_slider.value)
            with self.test_out:
                print(f"{self.select_subject.value} {action} pose 3d updated")
        except Exception as e:
            with self.test_out:
                print('update_pose_3d error', e)
               
    def update_frame_data(self, frame): # self.frame_num.value
        try: 
            with self.test_out:
                print(frame, self.frame_slider.value, end='\r')
            frame_num = frame-1
            #self.frame_input_2d = self.batch_input[frame_num].copy()
            self.pose_3d = self.pose_3d_list[frame_num].copy()
            self.dh_model.update_dh_model_from_pose(self.pose_3d)
            # updtate figure
            self.visualize_3d(frame_num)
        except Exception as e:
            with self.test_out:
                print(e)

    def visualize_3d(self, frame):
        try: 
            with self.plot_3d:
                plt.sca(self.ax_3d1)
                clear_axes(self.ax_3d1)
                #self.ax_3d1.set_title(f'frame {frame} GT - action: {self.select_action.value}')
                center_point = self.pose_3d[0].copy()
                #self.ax_3d1.set_xlim(center_point[0]-0.5, center_point[0]+0.5)
                #self.ax_3d1.set_ylim(center_point[1]-0.5, center_point[1]+0.5)
                #self.ax_3d1.set_zlim(center_point[2]-1, center_point[2]+1)
                self.ax_3d1.set_aspect('equal')
                draw_3d_pose(self.ax_3d1, self.pose_3d, alpha=0.5)
                self.dh_model.draw(self.ax_3d1, draw_frame=True, show_axis=True, scale=0.1, fontsize=5, show_angle=True)
        except Exception as e:
            with self.test_out:
                print(e)

    def visualize_angle(self, part):
        with self.test_out:
            print(f'visualize_angle {part}')
        try:
            if part == 'right_arm': part_id = self.right_arm_id
            elif part == 'left_arm': part_id = self.left_arm_id
            elif part == 'right_leg': part_id = self.right_leg_id
            elif part == 'left_leg': part_id = self.left_leg_id
            link1_azim = self.dh_angles[:, part_id*4+0]
            link1_elev = self.dh_angles[:, part_id*4+1]
            link2_azim = self.dh_angles[:, part_id*4+2]
            link2_elev = self.dh_angles[:, part_id*4+3]
            link1_azim_diff = self.dh_angles_diff[:, part_id*4+0]
            link1_elev_diff = self.dh_angles_diff[:, part_id*4+1]
            link2_azim_diff = self.dh_angles_diff[:, part_id*4+2]
            link2_elev_diff = self.dh_angles_diff[:, part_id*4+3]

            link1_azim_noise_frames = list(np.where(np.abs(link1_azim_diff) > np.pi/2)[0])
            link1_elev_noise_frames = list(np.where(np.abs(link1_elev_diff) > np.pi/2)[0])
            link2_azim_noise_frames = list(np.where(np.abs(link2_azim_diff) > np.pi/2)[0])
            link2_elev_noise_frames = list(np.where(np.abs(link2_elev_diff) > np.pi/2)[0])
            total_noise_frames = list(set(link1_azim_noise_frames + link1_elev_noise_frames + link2_azim_noise_frames + link2_elev_noise_frames))
            
            #noise_frames = list(set(list(np.where(np.abs(link1_azim_diff) > 1)[0]) + list(np.where(np.abs(link1_elev_diff) > 1)[0]) + list(np.where(np.abs(link2_azim_diff) > 1)[0]) + list(np.where(np.abs(link2_elev_diff) > 1)[0])))
            with self.test_out:
                print(f'noise frames: {total_noise_frames}')
                print(f'link1_azim noise frame: {link1_azim_noise_frames} {link1_azim[link1_azim_noise_frames]} {link1_azim[list(np.array(link1_azim_noise_frames)+np.ones_like(link1_azim_noise_frames))]}')
                print(f'link1_elev noise frame: {link1_elev_noise_frames} {link1_elev[link1_elev_noise_frames]} {link1_elev[list(np.array(link1_elev_noise_frames)+np.ones_like(link1_elev_noise_frames))]}')
                print(f'link2_azim noise frame: {link2_azim_noise_frames} {link2_azim[link2_azim_noise_frames]} {link2_azim[list(np.array(link2_azim_noise_frames)+np.ones_like(link2_azim_noise_frames))]}')
                print(f'link2_elev noise frame: {link2_elev_noise_frames} {link2_elev[link2_elev_noise_frames]} {link2_elev[list(np.array(link2_elev_noise_frames)+np.ones_like(link2_elev_noise_frames))]}')
                #print(np.where(link1_azim_diff>0)[0])

            with self.plot_angle:
                plt.sca(self.ax_angle)
                #clear_axes(self.ax_angle)
                self.ax_angle.clear()
                self.ax_angle.plot(np.arange(len(link1_azim)), link1_azim, 'r', label='link1_azim')
                self.ax_angle.plot(np.arange(len(link1_elev)), link1_elev, 'r--', label='link1_elev')
                self.ax_angle.plot(np.arange(len(link2_azim)), link2_azim, 'b', label='link2_azim')
                self.ax_angle.plot(np.arange(len(link2_elev)), link2_elev, 'b--', label='link2_elev')
                # self.ax_angle.plot(np.arange(len(link1_azim_diff)), link1_azim_diff, 'r', label='link1_azim_diff')
                # self.ax_angle.plot(np.arange(len(link1_elev_diff)), link1_elev_diff, 'r--', label='link1_elev_diff')
                # self.ax_angle.plot(np.arange(len(link2_azim_diff)), link2_azim_diff, 'b', label='link2_azim_diff')
                # self.ax_angle.plot(np.arange(len(link2_elev_diff)), link2_elev_diff, 'b--', label='link2_elev_diff')
                #self.ax_angle.plot(np.arange(len(self.right_arm_link1_azims_diff)), self.right_arm_link1_azims_diff, 'r', label='link1_azim_diff')
                #self.ax_angle.plot(np.arange(len(self.right_arm_link2_azims_diff)), self.right_arm_link2_azims_diff, 'b', label='link2_azim_diff')
                #self.ax_angle.plot(np.arange(len(self.right_arm_link2_azims)), np.rad2deg(self.right_arm_link2_azims), 'b', label='link2_azim_diff')
                self.ax_angle.set_xticks(np.arange(0, len(link1_azim), 50))
                self.ax_angle.set_ylim(-np.pi, np.pi)
                #self.ax_angle.set_ylim(-180, 180)
                self.ax_angle.grid()
                self.ax_angle.legend()
                
        except Exception as e:
            with self.test_out:
                print('here', e)

    def interactive(self): 
        self.load_dataset_interact = widgets.interactive_output(self.load_data, {'target_dataset': self.dataset_list})
        self.select_interact = widgets.interactive(self.update_select, subject=self.select_subject)
        self.update_pose_interact = widgets.interactive_output(self.update_pose_3d, {'subject': self.select_subject, 'action': self.select_action})
        self.frame_interact = widgets.interactive(self.update_frame_data, frame=self.frame_slider)
        self.part_interact = widgets.interactive(self.visualize_angle, part=self.select_part)

    def layout(self):
        ui_prepare = widgets.VBox([self.dataset_list, self.load_dataset_interact], layout=box_layout, width='auto')
        ui_select = widgets.VBox([self.select_subject, self.select_action, self.select_part], layout=box_layout)
        ui_figure = widgets.HBox([self.plot_angle, ui_select], layout=box_layout, width='auto')
        ui_level1 = widgets.HBox([ui_prepare, widgets.VBox([self.plot_3d, self.frame_slider, self.frame_button])])
        self.ui = widgets.VBox([ui_level1, ui_figure, self.test_out])

    def main(self, b=0):
        dis = display(self.ui)

ad = analysis_dataset()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
h36m_3d_world, h36m_cam_param = load_h36m()

==> Loading 3D data wrt World CS...


In [9]:
pose_3d_list = h36m_3d_world._data['S1']['Waiting 1']['positions'].copy()
frame_num = 717
dh_model = DHModel(pose_3d_list[frame_num].copy(), forward_dir='-z', use_global_frame=True)

1
d1 0.8832371181254449, d2 2.957608606262338 azim1 30.51, elev1 -40.86, azim2 -149.49, elev2 -139.14 prev_azim 0.00, prev_elev 0.00
d1 0.7220588867395102, d2 3.01257312229102 azim1 -29.51, elev1 29.33, azim2 150.49, elev2 150.67 prev_azim 0.00, prev_elev 0.00
2
d1 0.47702009037915866, d2 3.110451812327626 azim1 -26.19, elev1 7.88, azim2 153.81, elev2 172.12 prev_azim 0.00, prev_elev 0.00
d1 2.99280119039996, d2 2.8388317119629214 azim1 -171.38, elev1 -17.40, azim2 8.62, elev2 -162.60 prev_azim 0.00, prev_elev 0.00
3
d1 0.16783912911441812, d2 3.1356060615963726 azim1 4.68, elev1 8.40, azim2 -175.32, elev2 171.60 prev_azim 0.00, prev_elev 0.00
d1 0.18201796901104517, d2 3.1337227221516692 azim1 8.44, elev1 6.13, azim2 -171.56, elev2 173.87 prev_azim 0.00, prev_elev 0.00
4
d1 0.45165145459461026, d2 3.091702155733231 azim1 -16.54, elev1 -19.97, azim2 163.46, elev2 -160.03 prev_azim 0.00, prev_elev 0.00
d1 0.6681355410864316, d2 3.038951374652246 azim1 31.81, elev1 -21.58, azim2 -148.19,

In [10]:
dh_model.update_dh_model_from_pose(pose_3d_list[frame_num])
angles1 = dh_model.get_dh_angles(by_dict=False, degrees=True)
vector1 = dh_model.right_lower_arm_vector
dh_model.update_dh_model_from_pose(pose_3d_list[frame_num+1])
angles2 = dh_model.get_dh_angles(by_dict=False, degrees=True)
vector2 = dh_model.right_lower_arm_vector
angles1[2], angles1[3], angles2[2], angles2[3]

1
d1 0.0, d2 3.141592653589793 azim1 30.51, elev1 -40.86, azim2 -149.49, elev2 -139.14 prev_azim 30.51, prev_elev -40.86
d1 0.0, d2 3.141592653589793 azim1 -29.51, elev1 29.33, azim2 150.49, elev2 150.67 prev_azim -29.51, prev_elev 29.33
2
d1 0.0, d2 3.141592653589793 azim1 -26.19, elev1 7.88, azim2 153.81, elev2 172.12 prev_azim -26.19, prev_elev 7.88
d1 3.141592653589793, d2 0.0 azim1 -171.38, elev1 -17.40, azim2 8.62, elev2 -162.60 prev_azim 8.62, prev_elev -162.60
3
d1 0.0, d2 3.141592653589793 azim1 4.68, elev1 8.40, azim2 -175.32, elev2 171.60 prev_azim 4.68, prev_elev 8.40
d1 0.0, d2 3.141592653589793 azim1 8.44, elev1 6.13, azim2 -171.56, elev2 173.87 prev_azim 8.44, prev_elev 6.13
4
d1 0.0, d2 3.141592653589793 azim1 -16.54, elev1 -19.97, azim2 163.46, elev2 -160.03 prev_azim -16.54, prev_elev -19.97
d1 0.0, d2 3.141592653589793 azim1 31.81, elev1 -21.58, azim2 -148.19, elev2 -158.42 prev_azim 31.81, prev_elev -21.58
1
d1 0.0026240908250144686, d2 3.14132400445234 azim1 30.49,

(-29.508846940227244,
 29.32502921891796,
 -30.16489983566125,
 28.259941975318768)

In [10]:
vector1, vector2

(array([0.25045857, 0.01292384, 0.02169776], dtype=float32),
 array([0.24971816, 0.02879703, 0.01337516], dtype=float32))